In [148]:
import pickle

from metabolitics.preprocessing import MetaboliticsPipeline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate, StratifiedKFold

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, make_scorer

import numpy as np, pandas as pd
from collections import defaultdict, OrderedDict
from itertools import chain, starmap
from itertools import product

import matplotlib.pyplot as plt

In [3]:
results = pickle.load(open('../results/breast_cancer2.results','rb'))
labels = pickle.load(open('../datasets/breast_cancer2_y','rb'))

In [4]:
pipe = MetaboliticsPipeline(['reaction-diff',
                             'pathway_transformer'])

pre_processed_results = pipe.fit_transform(results, labels)

In [5]:
samples = defaultdict(lambda : [])
[
 samples[key].append(value) for key, value in 
 chain(*map(lambda sample: sample.items(), pre_processed_results))
]

dataset = pd.DataFrame(samples, index=labels)

In [6]:
dataset.T.head()

,unhealthy,unhealthy,unhealthy,healthy,unhealthy,unhealthy,unhealthy,unhealthy,healthy,healthy,...,unhealthy,healthy,healthy,unhealthy,unhealthy,unhealthy,unhealthy,unhealthy,healthy,unhealthy
,-25.490606,-21.389085,-29.843938,-1.452688e+01,-12.184960,4.338045,-7.707883,7.645137,99.308536,31.907804,...,-13.039948,-90.031445,-32.690873,-26.580457,-62.169126,-2.229793,-12.184928,-36.717150,-42.234431,42.581163
Alanine and aspartate metabolism,-100.777030,96.445184,38.294390,-1.007770e+02,6.018669,62.624693,24.222962,-100.777038,147.239393,-100.777038,...,-58.700946,24.222962,35.878663,24.222962,24.222962,24.222962,-100.777000,274.222962,-205.961401,15.846673
Alkaloid synthesis,0.000007,-0.000001,-0.000001,5.131799e-07,0.000006,-0.000001,-0.000001,-0.000001,-0.000001,-0.000001,...,0.000002,-0.000001,-0.000001,-0.000001,-0.000001,-0.000001,0.000038,-0.000001,-0.000001,-0.000001
Aminosugar metabolism,-55.414072,73.618178,256.413877,-4.466139e+01,-55.414075,-12.403328,-12.403328,-12.403328,-33.908704,116.628930,...,-55.414077,-55.414080,-12.403328,-55.414080,154.263339,84.370866,-55.414042,-55.414080,-33.908704,-33.908704
Androgen and estrogen synthesis and metabolism,0.000007,-0.000001,-0.000001,1.983698e-06,0.000006,-0.000001,-0.000001,-0.000001,-0.000001,-0.000001,...,0.000004,-0.000001,-0.000001,-0.000001,-0.000001,-0.000001,0.000038,-0.000001,-0.000001,-0.000001


In [8]:
balance = labels.count('unhealthy') / len(labels)

print(balance)

0.6477272727272727


In [9]:
binarize = lambda ls: np.array([1 if l == 'unhealthy' else 0 for l in ls])

X = dataset
y = binarize(dataset.index)

In [10]:
dataset.shape

(88, 100)

In [70]:
classifiers = [
  (LogisticRegression, {
    'C': np.geomspace(1e-6, 1e6, num=15),
    'max_iter': range(5, 30+1, 5)
  })
]

feature_selection = [

  (PCA, {
    'n_components': range(3, 21+1, 3)
  })
]

In [71]:
def build_pipeline(p):
    pipeline, pipeline_params = [], OrderedDict()
    
    for model, model_params in p:
        name = model.__name__
        
        pipeline.append((name, model()))
        pipeline_params.update({'{}__{}'.format(name, param_name) : values 
                                for param_name, values in model_params.items()})
    
    return Pipeline(pipeline), pipeline_params

### Nested cross-validation over 10 trials

In [120]:
NUM_TRIALS = 10
metrics = ['f1', 'recall', 'precision', 'accuracy']
trials = []

for i in range(NUM_TRIALS):
    cv_pipelines = []
    inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=i)
    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=i)
   
    for pipeline, params in map(build_pipeline, product(feature_selection, classifiers)):
        cv_pipeline = GridSearchCV(pipeline, params, cv=inner_cv, n_jobs=-1, verbose=1).fit(X, y)
        cv_pipelines.append(cv_pipeline)
        
    best_pipeline = cv_pipelines[np.argmax([i.best_score_ for i in cv_pipelines])]
    cv = cross_validate(best_pipeline.best_estimator_, 
                        X=X, y=y, cv=outer_cv, 
                        scoring=metrics, 
                        return_train_score=False)
    
    trials.append((best_pipeline, cv))

Fitting 10 folds for each of 630 candidates, totalling 6300 fits


[Parallel(n_jobs=-1)]: Done 356 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 1856 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 4356 tasks      | elapsed:   31.3s
[Parallel(n_jobs=-1)]: Done 6300 out of 6300 | elapsed:   45.2s finished


Fitting 10 folds for each of 630 candidates, totalling 6300 fits


[Parallel(n_jobs=-1)]: Done 356 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 1856 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 4356 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 6300 out of 6300 | elapsed:   47.0s finished


Fitting 10 folds for each of 630 candidates, totalling 6300 fits


...

[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 2368 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 4160 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done 6300 out of 6300 | elapsed:   52.4s finished


Fitting 10 folds for each of 630 candidates, totalling 6300 fits


[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 1696 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done 3856 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 5606 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 6285 out of 6300 | elapsed:   48.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 6300 out of 6300 | elapsed:   48.8s finished


Fitting 10 folds for each of 630 candidates, totalling 6300 fits


[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 5032 tasks      | elapsed:   33.0s
[Parallel(n_jobs=-1)]: Done 6300 out of 6300 | elapsed:   40.2s finished


### Trials stats!

In [170]:
trials_scores = [scores for model.best_estimator_, scores in trials]

In [171]:
trials_means = map(lambda trial_scores: {key: value.mean() 
                                         for key, value in trial_scores.items()}, trials_scores)

stats = pd.DataFrame(list(trials_means))

In [173]:
stats.mean()[2:]

test_accuracy     0.766111
test_f1           0.819688
test_precision    0.816405
test_recall       0.849000
dtype: float64